In [ ]:
# Install the Kaggle API package using pip
!pip install kaggle


In [ ]:
# Create the .kaggle directory in the home directory if it doesn't already exist
!mkdir -p ~/.kaggle

# Copy the kaggle.json file to the .kaggle directory
!cp kaggle.json ~/.kaggle/

# Set the file permissions to read and write for the owner only
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# Download the Sentiment140 dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile

# Path to the downloaded dataset zip file
dataset = "/content/sentiment140.zip"

# Extract the contents of the zip file
with ZipFile(dataset, 'r') as zip:
    zip.extractall()  # Extract all files to the current directory
    print("The dataset is extracted")  # Print a confirmation message


The datasert is extracted


In [ ]:
import numpy as np  # Import NumPy for numerical operations
import pandas as pd  # Import Pandas for data manipulation and analysis
import re  # Import re for regular expression operations
from nltk.corpus import stopwords  # Import stopwords from NLTK for text preprocessing
from nltk.stem.porter import PorterStemmer  # Import PorterStemmer for stemming words
from sklearn.feature_extraction.text import TfidfVectorizer  # Import TfidfVectorizer for converting text to TF-IDF features
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data into training and testing sets
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression for building the classification model
from sklearn.metrics import accuracy_score  # Import accuracy_score for evaluating model performance
import pickle  # Import pickle for saving and loading model objects


In [ ]:
import nltk

# Download the stopwords dataset from NLTK
nltk.download("stopwords")

# The stopwords dataset is now available for use in text preprocessing


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Print the list of English stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PREPROCESSING

In [ ]:
# Load Twitter data from the CSV file into a DataFrame
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1')

In [ ]:
# Print the shape of the DataFrame `twitter_data`
print(twitter_data.shape)


(1599999, 6)

In [ ]:
# Display the first few rows of the DataFrame `twitter_data`
print(twitter_data.head())


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# Define new column names
column_name=['target','ids','date','flag','user','text']


In [ ]:
# Load the dataset from a CSV file into a Pandas DataFrame
# Specify column names and encoding for reading the CSV file
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", names=column_name, encoding='ISO-8859-1')


In [ ]:
# Display a random sample of 5 rows from the DataFrame `twitter_data`
print(twitter_data.sample(5))


,target,ids,date,flag,user,text
233566,0,1979399313,Sun May 31 02:52:54 PDT 2009,NO_QUERY,chelliplum,i just found out i have a lot to be excited ab...
1053409,4,1961657328,Fri May 29 09:50:36 PDT 2009,NO_QUERY,djshelly,@scottkleinberg I was wondering if that was ho...
211576,0,1974524195,Sat May 30 13:18:31 PDT 2009,NO_QUERY,michelle_torres,@SunShine_713 idk and what about summer school
943112,4,1794911833,Thu May 14 06:48:13 PDT 2009,NO_QUERY,md_design,@hydro74 http://twitpic.com/55ets - YAY!!!! lo...
517035,0,2191210701,Tue Jun 16 04:46:40 PDT 2009,NO_QUERY,domness,@xStephTraff Ive been told you think I'm too n...


In [ ]:
# Replace all instances of 4 with 1 in the 'target' column
twitter_data['target'].replace({4:1},inplace=True)

In [ ]:
# Check for missing values in the DataFrame `twitter_data`
# Print the number of missing values for each column
print(twitter_data.isnull().sum())


,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# Importing the PorterStemmer class from the NLTK library for text stemming
port_stem = PorterStemmer()  # Creating an instance of the PorterStemmer class for stemming words

In [ ]:
def stemming(content):
    # Remove all non-alphabetic characters from the content and replace them with spaces
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)

    # Convert the content to lowercase
    stemmed_content = stemmed_content.lower()

    # Split the content into individual words
    stemmed_content = stemmed_content.split()

    # Stem each word and remove stopwords from the list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

    # Join the list of stemmed words back into a single string
    stemmed_content = ' '.join(stemmed_content)

    # Return the processed content
    return stemmed_content


In [ ]:
# Apply the stemming function to each entry in the 'text' column of the DataFrame and store the results in a new column 'stemmed_content'
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)


In [ ]:
# Display the first few rows of the DataFrame `twitter_data`
print(twitter_data.head())


,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
# Extract the values from the 'stemmed_content' column and assign them to X
X = twitter_data['stemmed_content'].values

# Extract the values from the 'target' column and assign them to Y
Y = twitter_data['target'].values


In [ ]:
# Print the feature vectors of the entire dataset
print(X)


['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
# Split the data into training and testing sets
# `test_size=0.2` specifies that 20% of the data will be used for testing
# `stratify=Y` ensures that the split maintains the same proportion of classes as in the original dataset
# `random_state=2` sets the seed for random number generation to ensure reproducibility of the split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [ ]:
# Print the shapes of the training and testing feature vectors and labels
print(X_train.shape)  # Shape of the training feature vectors
print(X_test.shape)   # Shape of the testing feature vectors
print(Y_train.shape)  # Shape of the training labels
print(Y_test.shape)   # Shape of the testing labels


((1280000,), (320000,), (1280000,), (320000,))

In [ ]:
# Initialize the TfidfVectorizer for transforming text data into TF-IDF feature vectors
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data and transform the training data into TF-IDF feature vectors
X_train = vectorizer.fit_transform(X_train)

# Transform the testing data into TF-IDF feature vectors using the fitted vectorizer
X_test = vectorizer.transform(X_test)


In [ ]:
# Import the LogisticRegression class from sklearn.linear_model
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
lr = LogisticRegression()

# Train the Logistic Regression model using the training data
lr.fit(X_train, Y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# Print the TF-IDF feature vectors for the training data
print(X_train)


  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [ ]:
# Print the TF-IDF feature vectors for the testing data
print(X_test)


  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

In [ ]:
# Predict the labels for the training data using the trained Logistic Regression model
X_train_predict = lr.predict(X_train)

# Calculate the accuracy of the model on the training data
train_accuracy = accuracy_score(Y_train, X_train_predict)

# Output the training accuracy
train_accuracy


0.80968203125

In [ ]:
# Predict the labels for the testing data using the trained Logistic Regression model
X_test_predict = lr.predict(X_test)

# Calculate the accuracy of the model on the testing data
test_accuracy = accuracy_score(Y_test, X_test_predict)

# Output the testing accuracy
test_accuracy


0.7781375

In [ ]:
# Save the trained Logistic Regression model to a file using pickle
pickle.dump(lr, open("twitter_model.pkl", 'wb'))
